# Tutorial 9
## MA course in Artificial Intelligence 2022/2023


In [1]:
import time
import numpy as np

## Min-Max and alpha-beta pruning for Tic-Tac-Toe game

In [2]:
class TicTacToeGame:
    # > The TicTacToeGame class is a class that represents a game of Tic Tac Toe
    def __init__(self):
        """
        The __init__ function initializes the game by setting the winner to None, the current state to an empty 3x3 grid,
        and the player turn to X.
        """
        self.winner = None
        self.current_state = np.array([['.', '.', '.'],
                                       ['.', '.', '.'],
                                       ['.', '.', '.']])
        # Player X always plays first
        self.player_turn = 'X'

    def draw_board(self):
        """
        It prints the current state of the board
        """
        for i in range(0, 3):
            for j in range(0, 3):
                print('{}|'.format(self.current_state[i][j]), end=" ")
            print()

    # Determines if the made move is a legal move
    def is_valid(self, px, py):
        """
        If the position is outside the board or the position is already taken, return False, otherwise return True

        :param px: The x coordinate of the position we want to check
        :param py: The y coordinate of the move
        :return: The function is_valid is returning a boolean value.
        """
        if px < 0 or px > 2 or py < 0 or py > 2:
            return False
        elif self.current_state[px][py] != '.':
            return False
        else:
            return True

    # Checks if the game has ended and returns the winner in each case
    def is_end(self):
        """
        If there's a winner, return the winner. If there's a tie, return a tie. If the game is still going, return None
        :return: The function is_end() returns the winner of the game.
        """
        # Vertical win
        for i in range(0, 3):
            if (self.current_state[0][i] != '.' and
                    self.current_state[0][i] == self.current_state[1][i] and
                    self.current_state[1][i] == self.current_state[2][i]):
                return self.current_state[0][i]

        # Horizontal win
        for i in range(0, 3):
            if self.current_state[i].tolist() == ['X', 'X', 'X']:
                return 'X'
            elif self.current_state[i].tolist() == ['O', 'O', 'O']:
                return 'O'

        # Main diagonal win
        if (self.current_state[0][0] != '.' and
                self.current_state[0][0] == self.current_state[1][1] and
                self.current_state[0][0] == self.current_state[2][2]):
            return self.current_state[0][0]

        # Second diagonal win
        if (self.current_state[0][2] != '.' and
                self.current_state[0][2] == self.current_state[1][1] and
                self.current_state[0][2] == self.current_state[2][0]):
            return self.current_state[0][2]

        # Is whole board full?
        for i in range(0, 3):
            for j in range(0, 3):
                # There's an empty field, we continue the game
                if self.current_state[i][j] == '.':
                    return None

        # It's a tie!
        return '.'

    def state(self):
        """
        If the game is over, return 'game over', otherwise return 'continue'
        :return: The state of the game.
        """
        self.winner = self.is_end()
        if self.winner in ["X", "O", "."]:
            return 'game over'
        return 'continue'

    def static_eval(self):
        """
        If X has won, return 1; if O has won, return -1; else, return 0
        :return: The static evaluation function returns 1 if the winner is X, -1 if the winner is O, and 0 if there is no
        winner.
        """
        if self.winner == "X":
            return 1
        elif self.winner == "O":
            return -1
        else:
            return 0

    def children(self):
        """
        For each possible move, create a new game state with the move applied, and yield that state
        """
        if self.player_turn == 'X':
            for i in range(3):
                for j in range(3):
                    if self.is_valid(i, j):
                        child = TicTacToeGame()
                        child.current_state = np.copy(self.current_state)
                        child.current_state[i][j] = "X"
                        child.player_turn = "O"
                        yield child

        elif self.player_turn == 'O':
            for i in range(3):
                for j in range(3):
                    if self.is_valid(i, j):
                        child = TicTacToeGame()
                        child.current_state = np.copy(self.current_state)
                        child.current_state[i][j] = "O"
                        child.player_turn = "X"
                        yield child

In [3]:
def play(game, strategy, alphabeta=False, to_plot=True):
    """
    It takes a game, a strategy, and a boolean value to determine whether to use alpha-beta pruning or not. It then plays
    the game until it's over, and returns the result

    :param game: the game object
    :param strategy: The strategy to be used
    :param alphabeta: True if you want to use alpha-beta pruning, False if you want to use minimax, defaults to False
    (optional)
    :param to_plot: If True, the board will be printed after each move, defaults to True (optional)
    :return: the state value of the current state.
    """
    remaining_pos = 9
    while True:
        alpha, beta = -2, 2
        result = game.is_end()
        if to_plot:
            game.draw_board()
            print()

        # Printing the appropriate message if the game has ended
        if result is not None:
            if to_plot:
                if result == 'X':
                    print('The winner is X!')
                elif result == 'O':
                    print('The winner is O!')
                elif result == '.':
                    print("It's a tie!")
            return

        qx, qy = None, None
        if game.player_turn == 'X':
            max_move = -2
            start_time = time.time()
            for i in range(3):
                for j in range(3):
                    if beta <= alpha:
                        break
                    if game.is_valid(i, j):
                        game.current_state[i][j] = "X"
                        game.player_turn = "O"
                        state_value = strategy(game, remaining_pos, True, alpha, beta)
                        game.current_state[i][j] = "."
                        game.player_turn = "X"
                        if state_value > max_move:
                            m, qx, qy = state_value, i, j
                            max_move = m
                        if alphabeta:
                            alpha = max(alpha, state_value)

                if beta <= alpha:
                    break

            end_time = time.time()

            if to_plot:
                print('Evaluation time: {} sec'.format(round(end_time - start_time, 3)))
                print('move X: X = {}, Y = {}'.format(qx, qy))
                if alphabeta:
                    print(f'alpha {alpha}, beta {beta}')
            game.current_state[qx][qy] = 'X'
            game.player_turn = 'O'
            remaining_pos -= 1

        else:
            min_move = 2
            start_time = time.time()
            for i in range(3):
                for j in range(3):
                    if beta <= alpha:
                        break
                    if game.is_valid(i, j):
                        game.current_state[i][j] = "O"
                        game.player_turn = "X"
                        state_value = strategy(game, remaining_pos, False, alpha, beta)
                        game.current_state[i][j] = "."
                        game.player_turn = "O"
                        if state_value < min_move:
                            m, qx, qy = state_value, i, j
                            min_move = m
                        if alphabeta:
                            beta = min(beta, state_value)

                if beta <= alpha:
                    break

            end_time = time.time()
            if to_plot:
                print('Evaluation time: {} sec'.format(round(end_time - start_time, 3)))
                print('move O: X = {}, Y = {}'.format(qx, qy))
                if alphabeta:
                    print(f'alpha {alpha}, beta {beta}')
            game.current_state[qx][qy] = 'O'
            game.player_turn = 'X'
            remaining_pos -= 1

## Min Max

In [4]:
def minimax(game, depth, maximizing_player, *args):
    """
    If we're at the end of the game, return the static evaluation of the current position.
    Otherwise, if we're the maximizing player, return the maximum of the static evaluations of the children.
    Otherwise, return the minimum of the static evaluations of the children.

    :param game: the game object
    :param depth: the depth of the search tree
    :param maximizing_player: True if the player is the maximizing player, False if the player is the minimizing player
    :return: The minimax function returns the best possible move for the player.
    """
    if depth == 0 or 'game over' in game.state():
        return game.static_eval()

    if maximizing_player:
        max_eval = -2
        for child in game.children():
            eval_position = minimax(child, depth - 1, maximizing_player=False)
            max_eval = max(max_eval, eval_position)
        return max_eval

    else:
        min_eval = 2
        for child in game.children():
            eval_position = minimax(child, depth - 1, maximizing_player=True)
            min_eval = min(min_eval, eval_position)
        return min_eval

In [5]:
g = TicTacToeGame()
play(g, minimax)

.| .| .| 
.| .| .| 
.| .| .| 

Evaluation time: 20.882 sec
move X: X = 0, Y = 0
X| .| .| 
.| .| .| 
.| .| .| 

Evaluation time: 2.33 sec
move O: X = 0, Y = 2
X| .| O| 
.| .| .| 
.| .| .| 

Evaluation time: 0.299 sec
move X: X = 1, Y = 0
X| .| O| 
X| .| .| 
.| .| .| 

Evaluation time: 0.033 sec
move O: X = 2, Y = 0
X| .| O| 
X| .| .| 
O| .| .| 

Evaluation time: 0.006 sec
move X: X = 1, Y = 1
X| .| O| 
X| X| .| 
O| .| .| 

Evaluation time: 0.001 sec
move O: X = 2, Y = 2
X| .| O| 
X| X| .| 
O| .| O| 

Evaluation time: 0.0 sec
move X: X = 1, Y = 2
X| .| O| 
X| X| X| 
O| .| O| 

The winner is X!


In [12]:
list_time = []

for _ in range(10):
    g = TicTacToeGame()
    start = time.time()
    play(g, minimax, to_plot=False)
    list_time.append(time.time() - start)

print("average time for min-max {:.3f} sec.".format(np.mean(list_time)))
print("std time for min-max {:.3f} sec.".format(np.std(list_time)))

average time for min-max 24.220 sec.
std time for min-max 1.020 sec.


# Alpha Beta pruning

In [8]:
def minimax_pruned(game, depth, maximizing_player, alpha, beta):
    """
    If the game is over, return the static evaluation of the game.
    Otherwise, if it's the maximizing player's turn, return the maximum of the static evaluations of the children, and if it's the minimizing player's turn, return the minimum of the static evaluations of the children.

    :param game: the game object
    :param depth: the depth of the search tree
    :param maximizing_player: True if the current player is the maximising player,
                              False if the current player is the minimising player
    :param alpha: the best value that the maximising player has found so far
    :param beta: the best value that the maximising player currently can guarantee at this point or above
    :return: The minimax_pruned function returns the evaluation of the position.
    """
    if depth == 0 or 'game over' in game.state():
        return game.static_eval()

    if maximizing_player:
        max_eval = -2
        for child in game.children():
            eval_position = minimax_pruned(child, depth - 1, False, alpha, beta)
            max_eval = max(max_eval, eval_position)
            alpha = max(alpha, eval_position)
            if beta <= alpha:
                break
        return max_eval

    else:
        min_eval = 2
        for child in game.children():
            eval_position = minimax_pruned(child, depth - 1, True, alpha, beta)
            min_eval = min(min_eval, eval_position)
            beta = min(beta, eval_position)
            if beta <= alpha:
                break
        return min_eval

In [9]:
# test on problem
g = TicTacToeGame()
play(g, minimax_pruned, alphabeta=True)

.| .| .| 
.| .| .| 
.| .| .| 

Evaluation time: 0.244 sec
move X: X = 0, Y = 0
alpha 1, beta 2
X| .| .| 
.| .| .| 
.| .| .| 

Evaluation time: 0.142 sec
move O: X = 0, Y = 2
alpha -2, beta 0
X| .| O| 
.| .| .| 
.| .| .| 

Evaluation time: 0.042 sec
move X: X = 1, Y = 0
alpha 1, beta 2
X| .| O| 
X| .| .| 
.| .| .| 

Evaluation time: 0.011 sec
move O: X = 2, Y = 0
alpha -2, beta -1
X| .| O| 
X| .| .| 
O| .| .| 

Evaluation time: 0.003 sec
move X: X = 1, Y = 1
alpha 1, beta 2
X| .| O| 
X| X| .| 
O| .| .| 

Evaluation time: 0.001 sec
move O: X = 2, Y = 2
alpha -2, beta -1
X| .| O| 
X| X| .| 
O| .| O| 

Evaluation time: 0.0 sec
move X: X = 1, Y = 2
alpha 1, beta 2
X| .| O| 
X| X| X| 
O| .| O| 

The winner is X!


In [11]:
list_time = []

for _ in range(10):
    g = TicTacToeGame()
    start = time.time()
    play(g, minimax_pruned, alphabeta=True, to_plot=False)
    list_time.append(time.time() - start)

print("average time for alpha-beta pruning {:.3f} sec.".format(np.mean(list_time)))
print("std time for alpha-beta pruning {:.3f} sec.".format(np.std(list_time)))

average time for alpha-beta pruning 0.451 seconds.
std time for alpha-beta pruning 0.062 seconds.
